In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window as W

In [2]:
spark = SparkSession.builder.appName('test_name').getOrCreate()

23/11/02 14:30:53 WARN Utils: Your hostname, MacBook.local resolves to a loopback address: 127.0.0.200; using 192.168.0.100 instead (on interface en0)
23/11/02 14:30:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/02 14:30:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option('inferSchema', 'true')\
.option('header', 'true').csv("/users/dznosok/downloads/owid-covid-data.csv")

In [33]:
df.printSchema()

root
 |-- iso_code: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- location: string (nullable = true)
 |-- date: date (nullable = true)
 |-- total_cases: double (nullable = true)
 |-- new_cases: double (nullable = true)
 |-- new_cases_smoothed: double (nullable = true)
 |-- total_deaths: double (nullable = true)
 |-- new_deaths: double (nullable = true)
 |-- new_deaths_smoothed: double (nullable = true)
 |-- total_cases_per_million: double (nullable = true)
 |-- new_cases_per_million: double (nullable = true)
 |-- new_cases_smoothed_per_million: double (nullable = true)
 |-- total_deaths_per_million: double (nullable = true)
 |-- new_deaths_per_million: double (nullable = true)
 |-- new_deaths_smoothed_per_million: double (nullable = true)
 |-- reproduction_rate: double (nullable = true)
 |-- icu_patients: double (nullable = true)
 |-- icu_patients_per_million: double (nullable = true)
 |-- hosp_patients: double (nullable = true)
 |-- hosp_patients_per_million: 

Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

In [7]:
df.select('iso_code', 'location', F.format_number((F.col('total_cases')/F.col('population')*100), 2)
          .alias('procent'))\
.where(F.col('date') == '2021-03-31')\
.orderBy(F.col('procent').desc())\
.show(15)

+--------+-------------+-------+
|iso_code|     location|procent|
+--------+-------------+-------+
|     LUX|   Luxembourg|   9.85|
|     ISR|       Israel|   9.63|
|     USA|United States|   9.20|
|     SRB|       Serbia|   8.83|
|     BHR|      Bahrain|   8.49|
|     PAN|       Panama|   8.23|
|     PRT|     Portugal|   8.06|
|     EST|      Estonia|   8.02|
|     SWE|       Sweden|   7.97|
|     LTU|    Lithuania|   7.94|
|     BEL|      Belgium|   7.61|
|     NLD|  Netherlands|   7.54|
|     GEO|      Georgia|   7.06|
|     ESP|        Spain|   7.02|
|     LIE|Liechtenstein|   7.00|
+--------+-------------+-------+
only showing top 15 rows



Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)

In [9]:
window = W.partitionBy("location").orderBy(F.col("new_cases").desc())

In [11]:
df.where(F.col('date') >= '2021-03-29').where(F.col('date') <= '2021-04-04')\
.where(F.col('continent') != 'null')\
.withColumn('rn', F.row_number().over(window))\
.where(F.col('rn')==1)\
.select('date', 'location', 'new_cases')\
.orderBy(F.col('new_cases').desc()).show(10)

+----------+-------------+---------+
|      date|     location|new_cases|
+----------+-------------+---------+
|2021-04-04|        India| 103558.0|
|2021-04-01|       Brazil|  91097.0|
|2021-04-04|       France|  80629.0|
|2021-04-01|United States|  79045.0|
|2021-04-03|       Turkey|  44756.0|
|2021-04-01|       Poland|  35253.0|
|2021-03-31|      Germany|  25014.0|
|2021-03-31|        Italy|  23887.0|
|2021-04-03|      Ukraine|  20456.0|
|2021-03-30|       Sweden|  16427.0|
+----------+-------------+---------+
only showing top 10 rows



Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в россии вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)



In [91]:
window = W.orderBy("date")

In [101]:
df.select( 'date', 'new_cases', F.col('new_cases') - F.col('new_cases'))\
.where(F.col('location') == 'Russia')\
.select('date', F.lag('new_cases').over(window).alias('lag'),'new_cases')\
.where(F.col('date') >= '2021-03-29').where(F.col('date') <= '2021-04-04')\
.select('date', 'lag', 'new_cases', (F.col('new_cases')-F.col('lag')).alias('delta')).show()

+----------+------+---------+------+
|      date|   lag|new_cases| delta|
+----------+------+---------+------+
|2021-03-29|8979.0|   8589.0|-390.0|
|2021-03-30|8589.0|   8162.0|-427.0|
|2021-03-31|8162.0|   8156.0|  -6.0|
|2021-04-01|8156.0|   9057.0| 901.0|
|2021-04-02|9057.0|   8682.0|-375.0|
|2021-04-03|8682.0|   8906.0| 224.0|
|2021-04-04|8906.0|   8697.0|-209.0|
+----------+------+---------+------+



23/10/31 22:47:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/31 22:47:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/31 22:47:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/31 22:47:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/31 22:47:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [12]:
spark.stop()